In [4]:
import os, sys, re, json, time, shutil
import itertools
import collections
from IPython.display import display

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf

# Pandas because pandas are awesome, and for pretty-printing
import pandas as pd
# Set pandas floating point display
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# Helper libraries for this notebook
import utils; reload(utils)
import vocabulary; reload(vocabulary)
import rnnlm; reload(rnnlm)

<module 'rnnlm' from 'rnnlm.pyc'>

In [5]:
import tensorflow as tf
import rnnlm; reload(rnnlm)

# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)


with tf.Graph().as_default():
  tf.set_random_seed(42)

  lm = rnnlm.RNNLM(V=10000, H=200, num_layers=2)
  lm.BuildCoreGraph()
  lm.BuildTrainGraph()
  lm.BuildSamplerGraph()

  summary_writer = tf.train.SummaryWriter("tf_summaries", 
                                          tf.get_default_graph())

In [6]:
toy_corpus = "<s> Mary had a little lamb . <s> The lamb was white as snow . <s>"
toy_corpus = np.array(toy_corpus.split())

print "Input words:"
bi = utils.batch_generator(toy_corpus, batch_size=2, max_time=4)
for i, (w,y) in enumerate(bi):
  utils.pretty_print_matrix(w, cols=["w_%d" % d for d in range(w.shape[1])], dtype=object)

print "Target words:"
bi = utils.batch_generator(toy_corpus, batch_size=2, max_time=4)
for i, (w,y) in enumerate(bi):
  utils.pretty_print_matrix(y, cols=["y_%d" % d for d in range(w.shape[1])], dtype=object)

Input words:


,w_0,w_1,w_2,w_3
0,<s>,Mary,had,a
1,<s>,The,lamb,was


,w_0,w_1,w_2
0,little,lamb,.
1,white,as,snow


Target words:


,y_0,y_1,y_2,y_3
0,Mary,had,a,little
1,The,lamb,was,white


,y_0,y_1,y_2
0,lamb,.,<s>
1,as,snow,.


In [7]:
def run_epoch(lm, session, batch_iterator, train=False,
              verbose=False, tick_s=10, 
              keep_prob=1.0, learning_rate=0.1):
  start_time = time.time()
  tick_time = start_time  # for showing status
  total_cost = 0.0  # total cost, summed over all words
  total_words = 0

  if train:
    train_op = lm.train_step_
    keep_prob = keep_prob
    loss = lm.train_loss_
  else:
    train_op = tf.no_op()
    keep_prob = 1.0  # no dropout at test time
    loss = lm.loss_  # true loss, if train_loss is an approximation

  for i, (w, y) in enumerate(batch_iterator):
    cost = 0.0
    #### YOUR CODE HERE ####
    np.random.seed(42)

    # At first batch in epoch, get a clean intitial state
    if i == 0:
        h = session.run(lm.initial_h_, {lm.input_w_: w})
 
    cost, h, _ = session.run([loss, lm.final_h_, train_op], feed_dict= {lm.target_y_: y, lm.initial_h_:h,
        lm.input_w_: w, lm.dropout_keep_prob_:keep_prob, lm.learning_rate_:learning_rate})      
    
    #### END(YOUR CODE) ####
    total_cost += cost
    total_words += w.size  # w.size = batch_size * max_time

    ##
    # Print average loss-so-far for epoch
    # If using train_loss_, this may be an underestimate.
    if verbose and (time.time() - tick_time >= tick_s):
      avg_cost = total_cost / total_words
      avg_wps = total_words / (time.time() - start_time)
      print "[batch %d]: seen %d words at %d wps, loss = %.3f" % (i,
          total_words, avg_wps, avg_cost)
      tick_time = time.time()  # reset time ticker

  return total_cost / total_words

In [8]:
# Load the dataset
V = 20000
vocab, train_ids, test_ids = utils.load_corpus("brown", split=0.8, V=V, shuffle=42)

Loaded 57340 sentences (1.16119e+06 tokens)
Training set: 45872 sentences (924077 tokens)
Test set: 11468 sentences (237115 tokens)


In [9]:
# Training parameters
max_time = 20
batch_size = 50
learning_rate = 0.5
keep_prob = 1.0
num_epochs = 5

# Model parameters
model_params = dict(V=V, 
                    H=100, 
                    num_layers=1)

trained_filename = 'tf_saved_rnnlm_trained'

In [10]:
def score_dataset(lm, session, ids, name="Data"):
  bi = utils.batch_generator(ids, batch_size=100, max_time=100)
  cost = run_epoch(lm, session, bi, 
                   learning_rate=1.0, keep_prob=1.0, 
                   train=False, verbose=False, tick_s=3600)
  print "%s: avg. loss: %.03f  (perplexity: %.02f)" % (name, cost, np.exp(cost))

In [11]:
# Will print status every this many seconds
print_interval = 5

# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)

with tf.Graph().as_default(), tf.Session() as session:
  # Seed RNG for repeatability
  tf.set_random_seed(42)
  
  with tf.variable_scope("model", reuse=None):
    lm = rnnlm.RNNLM(**model_params)
    lm.BuildCoreGraph()
    lm.BuildTrainGraph()
  
  session.run(tf.initialize_all_variables())
  saver = tf.train.Saver()
  
  for epoch in xrange(1,num_epochs+1):
    t0_epoch = time.time()
    bi = utils.batch_generator(train_ids, batch_size, max_time)
    print "[epoch %d] Starting epoch %d" % (epoch, epoch)
    #### YOUR CODE HERE ####

    run_epoch(lm, session, bi, train=True, keep_prob=keep_prob, learning_rate=learning_rate)

    #### END(YOUR CODE) ####
    print "[epoch %d] Completed in %s" % (epoch, utils.pretty_timedelta(since=t0_epoch))
    
    ##
    # score_dataset will run a forward pass over the entire dataset
    # and report perplexity scores. This can be slow (around 1/2 to 
    # 1/4 as long as a full epoch), so you may want to comment it out
    # to speed up training on a slow machine. Be sure to run it at the 
    # end to evaluate your score.
    print ("[epoch %d]" % epoch),
    score_dataset(lm, session, train_ids, name="Train set")
    print ("[epoch %d]" % epoch),
    score_dataset(lm, session, test_ids, name="Test set")
    print ""
    
    # Save a checkpoint
    saver.save(session, 'tf_saved_rnnlm', global_step=epoch)
    
  # Save final model
  saver.save(session, trained_filename)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


[epoch 1] Starting epoch 1
[epoch 1] Completed in 0:02:59
[epoch 1] Train set: avg. loss: 5.734  (perplexity: 309.34)
[epoch 1] Test set: avg. loss: 5.898  (perplexity: 364.17)

[epoch 2] Starting epoch 2
[epoch 2] Completed in 0:02:41
[epoch 2] Train set: avg. loss: 5.482  (perplexity: 240.29)
[epoch 2] Test set: avg. loss: 5.772  (perplexity: 321.03)

[epoch 3] Starting epoch 3
[epoch 3] Completed in 0:02:43
[epoch 3] Train set: avg. loss: 5.279  (perplexity: 196.13)
[epoch 3] Test set: avg. loss: 5.680  (perplexity: 292.85)

[epoch 4] Starting epoch 4
[epoch 4] Completed in 0:02:39
[epoch 4] Train set: avg. loss: 5.149  (perplexity: 172.23)
[epoch 4] Test set: avg. loss: 5.648  (perplexity: 283.72)

[epoch 5] Starting epoch 5
[epoch 5] Completed in 0:02:37
[epoch 5] Train set: avg. loss: 5.048  (perplexity: 155.65)
[epoch 5] Test set: avg. loss: 5.632  (perplexity: 279.11)



In [12]:
def sample_step(lm, session, input_w, initial_h):
  """Run a single RNN step and return sampled predictions.
  
  Args:
    lm : rnnlm.RNNLM
    session: tf.Session
    input_w : [batch_size] list of indices
    initial_h : [batch_size, hidden_dims]
  
  Returns:
    final_h : final hidden state, compatible with initial_h
    samples : [batch_size, 1] vector of indices
  """
  #### YOUR CODE HERE ####
  # Reshape input to column vector
  input_w = np.array(input_w, dtype=np.int32).reshape([-1,1])
  
  # Run sample ops
  final_h, samples = session.run([lm.final_h_, lm.pred_samples_], 
        feed_dict={lm.input_w_: input_w, lm.initial_h_: initial_h, lm.dropout_keep_prob_: 1.0, lm.learning_rate_:0.1})
  
  #### END(YOUR CODE) ####
  return final_h, samples[:,-1,:]

In [13]:
# Same as above, but as a batch
reload(rnnlm)
max_steps = 20
num_samples = 10
random_seed = 42

with tf.Graph().as_default(), tf.Session() as session:
  # Seed RNG for repeatability
  tf.set_random_seed(random_seed)

  with tf.variable_scope("model", reuse=None):
    lm = rnnlm.RNNLM(**model_params)
    lm.BuildCoreGraph()
    lm.BuildSamplerGraph()

  # Load the trained model
  saver = tf.train.Saver()
  saver.restore(session, trained_filename)

  # Make initial state for a batch with batch_size = num_samples
  w = np.repeat([[vocab.START_ID]], num_samples, axis=0)
  h = session.run(lm.initial_h_, {lm.input_w_: w})
  # We'll take one step for each sequence on each iteration 
  for i in xrange(max_steps):
    h, y = sample_step(lm, session, w[:,-1:], h)
    w = np.hstack((w,y))

  # Print generated sentences
  for row in w:
    for i, word_id in enumerate(row):
      print vocab.id_to_word[word_id],
      if (i != 0) and (word_id == vocab.START_ID):
        break
    print ""

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for tf_saved_rnnlm_trained
	 [[Node: save/RestoreV2_4 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_4/tensor_names, save/RestoreV2_4/shape_and_slices)]]

Caused by op u'save/RestoreV2_4', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 589, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 405, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/i

In [27]:
def score_seq(lm, session, seq, vocab):
  """Score a sequence of words. Returns total log-probability."""
  padded_ids = vocab.words_to_ids(utils.canonicalize_words(["<s>"] + seq, 
                                                           wordset=vocab.word_to_id))
  w = np.reshape(padded_ids[:-1], [1,-1])
  y = np.reshape(padded_ids[1:],  [1,-1])
  h = session.run(lm.initial_h_, {lm.input_w_: w})
  feed_dict = {lm.input_w_:w,
               lm.target_y_:y,
               lm.initial_h_:h,
               lm.dropout_keep_prob_: 1.0}
  # Return log(P(seq)) = -1*loss
  return -1*session.run(lm.loss_, feed_dict)

def load_and_score(inputs, sort=False):
  """Load the trained model and score the given words."""
  with tf.Graph().as_default(), tf.Session() as session:  
    with tf.variable_scope("model", reuse=None):
      lm = rnnlm.RNNLM(**model_params)
      lm.BuildCoreGraph()
        
    # Load the trained model
    saver = tf.train.Saver()
    saver.restore(session, trained_filename)
  
    if isinstance(inputs[0], str) or isinstance(inputs[0], unicode):
      inputs = [inputs]

    # Actually run scoring
    results = []
    for words in inputs:
      score = score_seq(lm, session, words, vocab)
      results.append((score, words))
    
    # Sort if requested
    if sort: results = sorted(results, reverse=True)
    
    # Print results
    for score, words in results:
      print "\"%s\" : %.05f" % (" ".join(words), score)

Now we can test as:

In [28]:
sents = ["once upon a time",
         "the quick brown fox jumps over the lazy dog"]
load_and_score([s.split() for s in sents])

"once upon a time" : -22.83544
"the quick brown fox jumps over the lazy dog" : -66.19430


In [30]:
#### YOUR CODE HERE ####
sents = ["the boy and the girl are",
         "the boy and the girl is",
         
         "the boys are", 
         "the boys is",
         
         "there are many deer", 
         "there are many deers"]
load_and_score([s.split() for s in sents])

#### END(YOUR CODE) ####

"the boy and the girl are" : -26.04387
"the boy and the girl is" : -26.93687
"the boys are" : -13.08485
"the boys is" : -15.46580
"there are many deer" : -21.82856
"there are many deers" : -14.51623


In [31]:
#### YOUR CODE HERE ####

sents = ["peanuts are my favorite kind of nut",
         "peanuts are my favorite kind of vegetable",
         
         "when I'm hungry I really prefer to eat",
         "when I'm hungry I really prefer to drink",
         
         "as nut",
         "as vegetable",
         
         "like nut", 
         "like vegetable",
         
         "peanut as nut",
         "peanut as vegetable"
         ]
load_and_score([s.split() for s in sents])


#### END(YOUR CODE) ####

"peanuts are my favorite kind of nut" : -44.99361
"peanuts are my favorite kind of vegetable" : -43.85052
"when I'm hungry I really prefer to eat" : -50.29917
"when I'm hungry I really prefer to drink" : -51.00814
"as nut" : -21.04525
"as vegetable" : -16.94885
"like nut" : -19.44227
"like vegetable" : -17.49213
"peanut as nut" : -23.27459
"peanut as vegetable" : -19.85958


In [32]:
prefix = "I have lots of".split()
noun = "toys"
adjectives = ["square", "green", "plastic"]
inputs = []
for adjs in itertools.permutations(adjectives):
  words = prefix + list(adjs) + [noun]
  inputs.append(words)
    
load_and_score(inputs, sort=True)

"I have lots of green square plastic toys" : -51.39275
"I have lots of square green plastic toys" : -51.87484
"I have lots of green plastic square toys" : -56.40881
"I have lots of plastic green square toys" : -58.53838
"I have lots of plastic square green toys" : -61.62395
"I have lots of square plastic green toys" : -62.61784


In [33]:
prefix = "I have lots of".split()
noun = "toys"
adjectives = ["flat", "green", "plastic"]
inputs = []
for adjs in itertools.permutations(adjectives):
  words = prefix + list(adjs) + [noun]
  inputs.append(words)
    
load_and_score(inputs, sort=True)

"I have lots of flat green plastic toys" : -51.23189
"I have lots of green flat plastic toys" : -53.88428
"I have lots of green plastic flat toys" : -57.95216
"I have lots of flat plastic green toys" : -61.01250
"I have lots of plastic green flat toys" : -61.71276
"I have lots of plastic flat green toys" : -62.24825


In [34]:
prefix = "I have lots of".split()
noun = "toys"
adjectives = ["round", "green", "plastic"]
inputs = []
for adjs in itertools.permutations(adjectives):
  words = prefix + list(adjs) + [noun]
  inputs.append(words)
    
load_and_score(inputs, sort=True)

"I have lots of round green plastic toys" : -52.98971
"I have lots of green round plastic toys" : -53.02505
"I have lots of green plastic round toys" : -55.81616
"I have lots of plastic round green toys" : -60.08774
"I have lots of round plastic green toys" : -61.78954
"I have lots of plastic green round toys" : -61.89275


In [36]:
prefix = "I have lots of".split()
noun = "toys"
adjectives = ["small", "green", "plastic"]
inputs = []
for adjs in itertools.permutations(adjectives):
  words = prefix + list(adjs) + [noun]
  inputs.append(words)
    
load_and_score(inputs, sort=True)

"I have lots of small green plastic toys" : -47.26843
"I have lots of green small plastic toys" : -49.53350
"I have lots of green plastic small toys" : -55.53246
"I have lots of plastic green small toys" : -57.74325
"I have lots of small plastic green toys" : -58.09218
"I have lots of plastic small green toys" : -58.98158
